In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [3]:
df = pd.read_csv('ch5_dataset.csv')

In [5]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [7]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [8]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [9]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [10]:
C = 1.0
n_splits = 5

In [11]:
#cross validation
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.840 +- 0.008


In [12]:
scores

[0.8420571311945114,
 0.845560604555688,
 0.8307445314067828,
 0.8301724275756219,
 0.8509773775532615]

In [14]:
#train final model

dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

I just found this markdown cheatsheet, https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet

This model still lives in our Jupyter Notebook.

How do we save it to basically export it?

# Save the Model by using pickle

In [15]:
import pickle

In [17]:
#take the model and write it to a file
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [18]:
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()

In [19]:
ls -lh *.bin

 Volume in drive C is Windows
 Volume Serial Number is 72FC-C6B1

 Directory of C:\Users\hp\Documents\Data Science\ch5


 Directory of C:\Users\hp\Documents\Data Science\ch5

11/02/2021  12:01 PM             2,416 model_C=1.0.bin
               1 File(s)          2,416 bytes
               0 Dir(s)  353,238,638,592 bytes free


In [20]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

Restarted the Kernel and are loading the model as if new

In [1]:
import pickle

In [2]:
input_file = 'model_C=1.0.bin'

In [3]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [4]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

# Saving the file to a py file

In [5]:
#In Juypter Notebook, go to /File /Download as / Python(.py)
# look at your downloads folder
# open py files in pycharm

In [ ]:
#He did a ton of cleanup stuff and went too fast
#One thing is to break apart the train and predict into two py files